<a href="https://colab.research.google.com/github/ARYANSRIVATSAVA/Projects/blob/main/N*M%20puzzle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
class State:
    
    
    final_cost = None
    minimum_weight_for_cost = None
    def __init__(self,goal, state, parent_state, path, depth, cost):
        
        self.goal = goal
        self.path = path
        self.state = state
        self.parent_state = parent_state
        self.depth = depth
        
        if parent_state:
            self.cost = parent_state.cost + cost

        else:
            self.cost = cost

            
            
    def check_optimal(self): 
        if self.state == self.goal:
            return True
        return False
        
    def Euclidean_Distance(self ,n,m): 
        self.minimum_weight_for_cost = 0
        for i in range(1 , n*m):
            euclidean_distance = pow(pow((self.state.index(i)), 2) + pow((self.goal.index(i)),2), 0.5)
          
            
            self.minimum_weight_for_cost = self.minimum_weight_for_cost + euclidean_distance/((n+m)/2) + euclidean_distance%((n+m)/2)
  
        self.final_cost = self.minimum_weight_for_cost + self.cost
        
        return(self.final_cost)
    
    
    def Manhattan_Distance(self ,n,m): 
        self.minimum_weight_for_cost = 0
        for i in range(1 , n*m):
            manhattan_distance = abs(self.state.index(i) - self.goal.index(i))
            
            
            self.minimum_weight_for_cost = self.minimum_weight_for_cost + manhattan_distance/((n+m)/2) + manhattan_distance%((n+m)/2)
    
        self.final_cost = self.minimum_weight_for_cost + self.cost
        
        return(self.final_cost)


   
    def Misplaced_Tiles(self,n,m): 
        counter = 0;
        self.minimum_weight_for_cost = 0
        for i in range(n*m):
            for j in range(n*m):
                if (self.state[i] != self.goal[j]):
                    counter += 1
                self.minimum_weight_for_cost = self.minimum_weight_for_cost + counter
   
        self.final_cost = self.minimum_weight_for_cost + self.cost

        return(self.final_cost)                
                    


    @staticmethod
    
 
    def available_directions(i,n,m): 
        directions = ['Left', 'Right', 'Up', 'Down']
        if i % m == 0:
            directions.remove('Left')
        if i % m == m-1:
            directions.remove('Right')
        if i - m < 0:
            directions.remove('Up')
        if i + m > n*m - 1:
            directions.remove('Down')

        return directions

    
    def generate_directions(self , n,m): 
        i = self.state.index(0)
        directions = self.available_directions(i,n,m)
        
        sub_configuration_list = []
        for path in directions:
            swap = self.state.copy()
            if path == 'Left':
                swap[i], swap[i - 1] = swap[i - 1], swap[i]
            elif path == 'Right':
                swap[i], swap[i + 1] = swap[i + 1], swap[i]
            elif path == 'Up':
                swap[i], swap[i - m] = swap[i - m], swap[i]
            elif path == 'Down':
                swap[i], swap[i + m] = swap[i + m], swap[i]
        
        
            sub_configuration_list.append(State(goal,swap, self, path, self.depth + 1, 1)) 
        return sub_configuration_list

    
    
    def optimal_solution(self):
        optimal_solution = self.state
        return optimal_solution

In [5]:

from queue import PriorityQueue
from queue import Queue
from queue import LifoQueue
checkpoints = []
positions = []

#Breadth-first Search
def BFS(goal,given_state , n,m):
    n_puzzle_initial_state = State(goal,given_state, None, None, 0, 0)
    if n_puzzle_initial_state.check_optimal():
        return n_puzzle_initial_state.optimal_solution()
    queue = Queue()
    queue.put(n_puzzle_initial_state)
    puzzle_configurations_list = []
    while not(queue.empty()):
        current_state = queue.get()
        puzzle_configurations_list.append(current_state.state)
        
        sub_configuration_list = current_state.generate_directions(n,m)
        for sub_configuration in sub_configuration_list:
            checkpoints.append(sub_configuration.state)
            #print(sub_configuration.state)
            for i in range(n*m):
                if(sub_configuration.state[i] == 0):
                    #print("index containing empty value: ",i)
                    x = int(i/m)
                    y = i%m
                    positions.append((x,y))
                    #print("position containing empty value: ",(x,y))
            if sub_configuration.state not in puzzle_configurations_list:
                if sub_configuration.check_optimal():
                    return sub_configuration.optimal_solution(), len(puzzle_configurations_list)
                queue.put(sub_configuration)
    return

#Depth-first Search 
def DFS(goal,given_state , n ,m): 
    n_puzzle_initial_state = State(goal,given_state, None, None, 0, 0)
    if n_puzzle_initial_state.check_optimal():
        return n_puzzle_initial_state.optimal_solution()
    queue = LifoQueue()
    queue.put(n_puzzle_initial_state)
    puzzle_configurations_list = []
    
    while not(queue.empty()):
        current_state = queue.get()
        max_depth = current_state.depth 
        puzzle_configurations_list.append(current_state.state)
        
        if max_depth == 30:
            continue 

        sub_configuration_list = current_state.generate_directions(n,m)
        for sub_configuration in sub_configuration_list:
            checkpoints.append(sub_configuration.state)
            #print(sub_configuration.state)
            for i in range(n*m):
                if(sub_configuration.state[i] == 0):
                    #print("index containing empty value: ",i)
                    x = int(i/m)
                    y = i%m
                    positions.append((x,y))
                    #print("position containing empty value: ",(x,y))
            if sub_configuration.state not in puzzle_configurations_list:
                if sub_configuration.check_optimal():
                    return sub_configuration.optimal_solution(), len(puzzle_configurations_list)
                queue.put(sub_configuration)
    return (("Couldn't find optimal_solution in the limited depth."), len(puzzle_configurations_list))
        
    

def AStar_Euclidean(goal,given_state , n,m):
    queue = PriorityQueue()
    puzzle_configurations_list = []
    counter = 0
    n_puzzle_initial_state = State(goal,given_state, None, None, 0, 0)
    get_state = n_puzzle_initial_state.Euclidean_Distance(n,m)
    queue.put((get_state, counter, n_puzzle_initial_state))

    while not queue.empty():
        current_state = queue.get()
        current_state = current_state[2]
        puzzle_configurations_list.append(current_state.state)
        
        if current_state.check_optimal():
            return current_state.optimal_solution(), len(puzzle_configurations_list)

        sub_configuration_list = current_state.generate_directions(n,m)
        for sub_configuration in sub_configuration_list:
            checkpoints.append(sub_configuration.state)
            #print(sub_configuration.state)
            for i in range(n*m):
                if(sub_configuration.state[i] == 0):
                    #print("index containing empty value: ",i)
                    x = int(i/m)
                    y = i%m
                    positions.append((x,y))
                    #print("position containing empty value: ",(x,y))
            if sub_configuration.state not in puzzle_configurations_list:
                counter += 1
                get_state = sub_configuration.Euclidean_Distance(n,m) 
                queue.put((get_state, counter, sub_configuration)) 
    return




def AStar_Manhattan(goal,given_state , n,m):
    queue = PriorityQueue()
    puzzle_configurations_list = []
    counter = 0
    n_puzzle_initial_state = State(goal,given_state, None, None, 0, 0)
    get_state = n_puzzle_initial_state.Manhattan_Distance(n,m)
    queue.put((get_state, counter, n_puzzle_initial_state))  

    while not queue.empty():
        current_state = queue.get()
        current_state = current_state[2]
        puzzle_configurations_list.append(current_state.state)
        
        if current_state.check_optimal():
            return current_state.optimal_solution(), len(puzzle_configurations_list)

        sub_configuration_list = current_state.generate_directions(n,m)
        for sub_configuration in sub_configuration_list:
            checkpoints.append(sub_configuration.state)
            #print(sub_configuration.state)
            for i in range(n*m):
                if(sub_configuration.state[i] == 0):
                    #print("index containing empty value: ",i)
                    x = int(i/m)
                    y = i%m
                    positions.append((x,y))
                    #print("position containing empty value: ",(x,y))
            if sub_configuration.state not in puzzle_configurations_list:
                counter += 1
                get_state = sub_configuration.Manhattan_Distance(n,m) 
                queue.put((get_state, counter, sub_configuration)) 
    return


def AStar_Misplaced(goal,given_state , n,m):
    queue = PriorityQueue()
    puzzle_configurations_list = []
    counter = 0
    n_puzzle_initial_state = State(goal,given_state, None, None, 0, 0)
    get_state = n_puzzle_initial_state.Misplaced_Tiles(n,m) 
    queue.put((get_state, counter, n_puzzle_initial_state)) 

    while not queue.empty():
        current_state = queue.get()
        current_state = current_state[2]
        puzzle_configurations_list.append(current_state.state)
        
        if current_state.check_optimal():
            return current_state.optimal_solution(), len(puzzle_configurations_list)

        sub_configuration_list = current_state.generate_directions(n,m)
        for sub_configuration in sub_configuration_list:
            checkpoints.append(sub_configuration.state)
            #print(sub_configuration.state)
            for i in range(n*m):
                if(sub_configuration.state[i] == 0):
                    #print("index containing empty value: ",i)
                    x = int(i/m)
                    y = i%m
                    positions.append((x,y))
                    #print("position containing empty value: ",(x,y))
            if sub_configuration.state not in puzzle_configurations_list:
                counter += 1
                get_state = sub_configuration.Misplaced_Tiles(n,m) 
                queue.put((get_state, counter, sub_configuration)) 
    return    

In [6]:
from ctypes import c_int32
print(" Enter these numbers for the select the following methods -> BFS: 1, DFS: 2, A*_0: 3, A*_1: 4, A*_2: 5" ) 
method = int(input("Enter method\n")) 
tmax = float(input("Enter tmax\n"))
n = int(input("Enter n\n"))
m = int(input("Enter m\n"))

#goal = [1,2,3,4,5,6,7,8,0]
print("Enter your" ,n,"*",m, "check_function")
n_puzzle_initial_state = []
for i in range(0,n*m):
    p = int(input())
    n_puzzle_initial_state.append(p)

print("The given state is:", n_puzzle_initial_state)

goal = []
for i in range(n*m):
   j = int(input("Enter goal state values "))
   goal.append(j)

        
def inversions_count(state):
    cnt = 0
    for x in range(len(state)-1):
        for y in range(i+1 , len(state)):
            if (( state[x] > state[y]) and state[x] and state[y]):
                cnt += 1
    return cnt

def check_function(state): 
    counter = inversions_count(state)
    if (counter %2 ==0):
        return True
    return False

 

        
from time import time
 
if check_function(n_puzzle_initial_state):
    print("solvable, please wait. \n")
    
    with open("solutions.txt", "w") as file:
    
    
        if(method == 1):
           time1 = time()
           BFS_output = BFS(goal,n_puzzle_initial_state, n , m)
           BFS_time = time() - time1
           x = BFS_output[1]
           for i in range(len(checkpoints)):
                print(checkpoints[i])
                print(positions[i])
                file.write(f'{checkpoints[i]}\n')
                file.write(f'{positions[i]}\n')
           if BFS_time <= tmax:
             
              print('Number of puzzle_configurations_list nodes is ', BFS_output[1])  
              file.write(f'Number of puzzle_configurations_list nodes is {BFS_output[1]} \n')  
              print('BFS Time:', BFS_time , "\n")
              file.write(f'BFS Time: {BFS_time} \n')
              
           else:
              print("optimal_solution cannot be determined in given time")
              file.write('optimal_solution cannot be determined in given time')

        if(method == 2):
            time2 = time()
            DFS_output = DFS(goal,n_puzzle_initial_state, n ,m )
            DFS_time = time() - time2
            x = DFS_output[1]
            for i in range(len(checkpoints)):
                print(checkpoints[i])
                print(positions[i])
                file.write(f'{checkpoints[i]}\n')
                file.write(f'{positions[i]}\n')
            if DFS_time <= tmax:
             
              print('Number of puzzle_configurations_list nodes is ', DFS_output[1])  
              file.write(f'Number of puzzle_configurations_list nodes is {DFS_output[1]} \n')  
              print('BFS Time:', DFS_time , "\n")
              file.write(f'BFS Time: {DFS_time} \n')
              
            else:
              print("optimal_solution cannot be determined in given time")
              file.write('optimal_solution cannot be determined in given time')
        if(method == 3):
            time3 = time()
            AStar_output = AStar_Euclidean(goal,n_puzzle_initial_state, n , m)
            euclidean_time = time() - time3
            x = AStar_output[1]
            for i in range(len(checkpoints)):
                print(checkpoints[i])
                print(positions[i])
                file.write(f'{checkpoints[i]}\n')
                file.write(f'{positions[i]}\n')
            if euclidean_time <= tmax:
             
              print('Number of puzzle_configurations_list nodes is ', AStar_output[1])  
              file.write(f'Number of puzzle_configurations_list nodes is {AStar_output[1]} \n')  
              print('BFS Time:', euclidean_time , "\n")
              file.write(f'BFS Time: {euclidean_time} \n')
              
            else:
              print("optimal_solution cannot be determined in given time")
              file.write('optimal_solution cannot be determined in given time')


        if(method == 4):
            time4 = time()
            AStar_manhattan_output = AStar_Manhattan(goal,n_puzzle_initial_state, n , m)
            manhattan_time = time() - time4
            for i in range(len(checkpoints)):
                print(checkpoints[i])
                print(positions[i])
                file.write(f'{checkpoints[i]}\n')
                file.write(f'{positions[i]}\n')
            if manhattan_time <= tmax:
             
              print('Number of puzzle_configurations_list nodes is ', AStar_manhattan_output[1])  
              file.write(f'Number of puzzle_configurations_list nodes is {AStar_manhattan_output[1]} \n')  
              print('BFS Time:', manhattan_time , "\n")
              file.write(f'BFS Time: {manhattan_time} \n')
              
            else:
              print("optimal_solution cannot be determined in given time")
              file.write('optimal_solution cannot be determined in given time')
 

        if(method == 5):
            time5 = time()
            AStar_misplaced_output = AStar_Misplaced(goal,n_puzzle_initial_state, n , m)
            misplaced_time = time() - time5
            x = AStar_misplaced_output[1]
            for i in range(len(checkpoints)):
                print(checkpoints[i])
                print(positions[i])
                file.write(f'{checkpoints[i]}\n')
                file.write(f'{positions[i]}\n')
            if misplaced_time <= tmax:
             
              print('Number of puzzle_configurations_list nodes is ', AStar_misplaced_output[1])  
              file.write(f'Number of puzzle_configurations_list nodes is {AStar_misplaced_output[1]} \n')  
              print('BFS Time:', misplaced_time , "\n")
              file.write(f'BFS Time: {misplaced_time} \n')
               
            else:
              print("optimal_solution cannot be determined in given time")
              file.write('optimal_solution cannot be determined in given time')

    
    
else:
  
    print("Not solvable")



 Enter these numbers for the select the following methods -> BFS: 1, DFS: 2, A*_0: 3, A*_1: 4, A*_2: 5
Enter method
4
Enter tmax
3
Enter n
3
Enter m
3
Enter your 3 * 3 check_function
1
8
2
0
4
3
7
6
5
The given state is: [1, 8, 2, 0, 4, 3, 7, 6, 5]
Enter goal state values 1
Enter goal state values 2
Enter goal state values 3
Enter goal state values 4
Enter goal state values 5
Enter goal state values 6
Enter goal state values 7
Enter goal state values 8
Enter goal state values 0
solvable, please wait. 

[1, 8, 2, 4, 0, 3, 7, 6, 5]
position containing empty value:  (1, 1)
[0, 8, 2, 1, 4, 3, 7, 6, 5]
position containing empty value:  (0, 0)
[1, 8, 2, 7, 4, 3, 0, 6, 5]
position containing empty value:  (2, 0)
[1, 8, 2, 0, 4, 3, 7, 6, 5]
position containing empty value:  (1, 0)
[1, 8, 2, 4, 3, 0, 7, 6, 5]
position containing empty value:  (1, 2)
[1, 0, 2, 4, 8, 3, 7, 6, 5]
position containing empty value:  (0, 1)
[1, 8, 2, 4, 6, 3, 7, 0, 5]
position containing empty value:  (2, 1)
[1, 8, 2,

In [ ]:
25